In [3]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tslearn.metrics import dtw, soft_dtw, dtw_path
from sklearn.metrics import accuracy_score
from scipy.spatial import distance as dist
from scipy.stats import norm

from reader import get_x_label, get_class_x
from average import amplitude_avg, dtw_avg, soft_dtw_avg

In [4]:
file_name = 'Gun_Point'

In [5]:
class_x = get_class_x(file_name + '_TRAIN')
class_avg = dtw_avg(class_x)

In [6]:
dtw_stat = [[list() for i in range(len(class_x[0][0]))] for _ in range(len(class_x))]
for i in range(class_x.shape[0]):
    for j in range(len(class_x[i])):
        path, dist = dtw_path(class_avg[i], class_x[i][j])
        for avg_pos, x_pos in path:
            dtw_stat[i][avg_pos].append(class_x[i][j][x_pos])
dtw_stat = np.array(dtw_stat)
for i in range(dtw_stat.shape[0]):
    for j in range(dtw_stat.shape[1]):
        dtw_stat[i] = np.array(dtw_stat[i])

In [7]:
dtw_mean = [[list() for i in range(dtw_stat.shape[1])] for _ in range(dtw_stat.shape[0])]
dtw_sd = [[list() for i in range(dtw_stat.shape[1])] for _ in range(dtw_stat.shape[0])]
for class_label in range(dtw_stat.shape[0]):
    for point in range(dtw_stat.shape[1]):
        stat = np.array(dtw_stat[class_label][point])
        dtw_mean[class_label][point].append(stat.mean())
        dtw_sd[class_label][point].append(stat.std())

In [8]:
test_x, test_label = get_x_label(file_name + '_TEST')

In [10]:
bay_predict = []
dist_predict = []
EPSILON = 10**(-10)
count = 0
for x in test_x:
    min_log_prob = 10**(310)
    min_dist = 100000
    bay_nearest_class = -1
    dist_nearest_class = -1
    for j in range(len(class_avg)):
        # Class j
        path, dist = dtw_path(x, class_avg[j])
        log_prob = 1
        probs = [list() for _ in range(len(x))]
        for xx in x:
            area_up = norm(dtw_mean[j][avg_pos][0], dtw_sd[j][avg_pos][0]).cdf(xx)
            area_low = norm(dtw_mean[j][avg_pos][0], dtw_sd[j][avg_pos][0]).cdf(xx - EPSILON)
            prob = area_up - area_low
            if prob < 10**(-310):
                prob = 10**(-310)
            log_prob += -np.log(prob)
        if log_prob < min_log_prob:
            min_log_prob = log_prob
            bay_nearest_class = j
        if dist < min_dist:
            min_dist = dist
            dist_nearest_class = j
    bay_predict.append(bay_nearest_class)
    dist_predict.append(dist_nearest_class)
    print(count, ':', bay_nearest_class, dist_nearest_class, test_label[count]-1)
    count += 1

0 : 0 1 0
1 : 0 1 1
2 : 1 1 1
3 : 1 0 0
4 : 0 0 0
5 : 0 1 1
6 : 1 0 0
7 : 1 1 1
8 : 0 0 1
9 : 0 0 0
10 : 0 0 0
11 : 0 0 0
12 : 0 1 1
13 : 0 0 0
14 : 1 0 0
15 : 0 0 0
16 : 0 1 0
17 : 0 1 1
18 : 0 1 1
19 : 0 1 1
20 : 0 0 0
21 : 1 1 1
22 : 0 0 0
23 : 1 0 0
24 : 0 0 0
25 : 1 0 1
26 : 1 0 0
27 : 1 1 0
28 : 0 0 1
29 : 1 0 0
30 : 0 1 0
31 : 1 1 1
32 : 1 0 1
33 : 0 1 0
34 : 1 0 0
35 : 0 0 0
36 : 0 0 0
37 : 0 0 0
38 : 0 0 0
39 : 0 0 1
40 : 0 0 0
41 : 1 1 1
42 : 0 0 0
43 : 0 0 0
44 : 1 1 1
45 : 0 1 0
46 : 1 1 1
47 : 0 0 1
48 : 1 1 0
49 : 0 0 1
50 : 0 1 1
51 : 1 0 1
52 : 1 0 0
53 : 1 1 1
54 : 0 0 0
55 : 0 0 0
56 : 1 0 1
57 : 0 1 1
58 : 1 0 0
59 : 0 1 1
60 : 0 0 0
61 : 1 1 1
62 : 0 0 0
63 : 0 0 0
64 : 1 0 1
65 : 0 0 0
66 : 0 0 0
67 : 0 0 1
68 : 0 1 1
69 : 1 1 1
70 : 1 0 1
71 : 0 1 1
72 : 0 0 1
73 : 0 0 1
74 : 0 0 1
75 : 0 1 1
76 : 0 1 1
77 : 1 1 1
78 : 1 1 1
79 : 0 0 0
80 : 1 0 0
81 : 0 0 0
82 : 1 1 1
83 : 1 1 1
84 : 0 1 1
85 : 0 1 1
86 : 1 0 0
87 : 0 0 1
88 : 0 0 0
89 : 0 0 0
90 : 0 1 0
91 : 0 0 

In [11]:
accuracy_score(test_label-1, bay_predict)

0.5933333333333334